In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# 基于负采样的skip-gram模型
class SkipGramModel(nn.Module):
    # embed_size是词表大小，embed_dimension是词向量维度
    def __init__(self,embed_size,embed_dimension):
        super(SkipGramModel,self).__init__()
        self.embed_size = embed_size
        self.embed_dimension = embed_dimension
        self.w_embeddings = nn.Embedding(embed_size,embed_dimension,sparse=True) # 一行一个词向量，中心词
        self.v_embeddings = nn.Embedding(embed_size, embed_dimension, sparse=True)  #周围词
        self._init_emb()

    #官方代码中的初始化方法
    def _init_emb(self):
        initrange = 0.5 / self.embed_dimension
        self.w_embeddings.weight.data.uniform_(-initrange, initrange)# 这是个正态分布，中心词
        self.v_embeddings.weight.data.uniform_(-0, 0)#这个是周围词
    
    # pow_w是中心词，pos_v是周围词，他们两个一一组合在一起是一个正样本，负样本是一个中心词对应三个随机采样的周围词
    def forward(self, pos_w, pos_v, neg_v):
        #这个是中心词的词向量，pos_w是batch_size*词向量维度大小的词向量
        emb_w = self.w_embeddings(torch.LongTensor(pos_w).cuda())  # 转为tensor 大小 [ mini_batch_size * emb_dimension ]
        #这是周围词，同意是
        emb_v = self.v_embeddings(torch.LongTensor(pos_v).cuda())
        #这是负样本batch_size*词向量维度，batch_size*负样本个数（这里是3）*词向量维度
        neg_emb_v = self.v_embeddings(torch.LongTensor(neg_v).cuda())  # 转换后大小 [ mini_batch_size * negative_sampling_number * emb_dimension ]
        score = torch.mul(emb_w, emb_v).squeeze() #损失函数，中心词和周围词相乘,.squeeze()是去掉取值为1的维度，这个是对应元素
                                #相乘，最后得到的大小仍然是batch_size*词向量维度大小

        score = torch.sum(score, dim=1) #上面是对应维度相乘，这里是把他加起来，等同于内积的效果
        score = torch.clamp(score, max=10, min=-10) #限定大小范围
        score = F.logsigmoid(score)

        neg_score = torch.bmm(neg_emb_v, emb_w.unsqueeze(2)).squeeze() #增加了一个维度，emb_w变成batch_size*词向量维度*1,bmm
                                          #是两个三维的向量相乘
        neg_score = torch.clamp(neg_score, max=10, min=-10) 
        neg_score = F.logsigmoid(-1 * neg_score) # 负采样的分数越小越好
        # L = log sigmoid (Xw.T * θv) + ∑neg(v) [log sigmoid (-Xw.T * θneg(v))]
        loss = - torch.sum(score) - torch.sum(neg_score)
        return loss

    #词向量的保存
    def save_embedding(self, id2word, file_name):
        embedding_1 = self.w_embeddings.weight.data.cpu().numpy()
        embedding_2 = self.v_embeddings.weight.data.cpu().numpy()
        embedding = (embedding_1+embedding_2)/2 #把两个矩阵求和除2
        print(embedding)

In [6]:
model = SkipGramModel(100, 10)
id2word = dict()
for i in range(100):
    id2word[i] = str(i)
pos_w = [0, 0, 1, 1, 1] #中心词
pos_v = [1, 2, 0, 2, 3] #真正的周围词
neg_v = [[23, 42, 32], [32, 24, 53], [32, 24, 53], [32, 24, 53], [32, 24, 53]] #一个中心词对应三个周围词的负样本
model.forward(pos_w, pos_v, neg_v)


RuntimeError: ignored

In [ ]:
model.w_embeddings.weight.data.cpu().numpy().shape